In [98]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
DATADIR = Path("data")

In [100]:
from data_processing import *
import data_processing as dp

from train import *

In [101]:
partitions = []
partition_list = [
    0
    if i in range(0, 4)
    else 1
    if i in range(5, 14 + 1)
    else 2
    if i in range(15, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})

partition_list = [
    0
    if i == 0
    else 1
    if i in range(1, 9 + 1)
    else 2
    if i in range(10, 19 + 1)
    else 3
    if i in range(20, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})


partition_list = [
    0
    if i in range(0, 4)
    else 1
    if i in range(5, 9 + 1)
    else 2
    if i in range(10, 14 + 1)
    else 3
    if i in range(15, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})

partitions.append(None)


partition_list = [
    1
    if i in range(1, 4 + 1)
    else 2
    if i in range(5, 9 + 1)
    else 3
    if i in range(10, 14 + 1)
    else 4
    if i in range(15, 19 + 1)
    else 5
    if i in range(20, 27 + 1)
    else 0
    for i in range(0, 27 + 1)
]
partitions.append({i: v for i, v in enumerate(partition_list)})

# Trying dailies additively

In [102]:
def all_combinations_scores(dailies_names=[]):
    train_scores = []
    test_scores = []
    train_scores_sel = []
    test_scores_sel = []
    splits_by_participant = []
    test_take_first = []

    # No split by participant
    SPLIT_BY_PARTICIPANT = False
    for partition in partitions:
        res = train(
            TYPE="classification",
            TARGET="value",
            SPLIT_BY_PARTICIPANT=SPLIT_BY_PARTICIPANT,
            TEST_TAKE_FIRST=-1,
            SEED=550,
            return_csv=False,
            feature_selection=True,
            partition=partition,
            verbose=False,
            dailies_names=dailies_names,
        )
        train_scores.append(res[0])
        test_scores.append(res[1])
        train_scores_sel.append(res[2])
        test_scores_sel.append(res[3])
        splits_by_participant.append(SPLIT_BY_PARTICIPANT)
        test_take_first.append(0)

    # Split by participant, take first 0,1,2,3,4
    SPLIT_BY_PARTICIPANT = True
    for TEST_TAKE_FIRST in range(0, 4 + 1):
        for partition in partitions:
            res = train(
                TYPE="classification",
                TARGET="value",
                SPLIT_BY_PARTICIPANT=SPLIT_BY_PARTICIPANT,
                TEST_TAKE_FIRST=TEST_TAKE_FIRST,
                SEED=550,
                return_csv=False,
                feature_selection=True,
                partition=partition,
                verbose=False,
                dailies_names=dailies_names,
            )
            train_scores.append(res[0])
            test_scores.append(res[1])
            train_scores_sel.append(res[2])
            test_scores_sel.append(res[3])
            splits_by_participant.append(SPLIT_BY_PARTICIPANT)
            test_take_first.append(TEST_TAKE_FIRST)

    df = pd.DataFrame(
        {
            "partition": [
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
            ],
            "train": train_scores,
            "test": test_scores,
            "train (fs)": train_scores_sel,
            "test (fs)": test_scores_sel,
            "SPLIT": splits_by_participant,
            "TTF": test_take_first,
        }
    )
    dailies_substr = "_".join([str(x) for x in dailies_names])
    df.to_csv("partition_results_" + dailies_substr + ".csv", index=False)

    return df

## Locations (base)

In [103]:
df_loc = pd.read_csv("partition_results_.csv")
df_loc

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",96.855984,70.857143,96.653144,71.428571,False,0
1,"0, 1-9, 10-19, 20-27",96.450304,69.714286,96.146045,66.285714,False,0
2,"0-4, 5-9, 10-14, 15-27",95.740365,57.142857,95.131846,59.428571,False,0
3,base,94.523327,50.285714,94.523327,53.714286,False,0
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",95.233266,45.714286,94.523327,45.714286,False,0
5,"0-4, 5-14, 15-27",96.800826,56.250000,96.594427,60.416667,True,0
6,"0, 1-9, 10-19, 20-27",96.284830,46.875000,95.975232,45.833333,True,0
7,"0-4, 5-9, 10-14, 15-27",94.943240,39.583333,94.530444,39.062500,True,0
8,base,94.117647,26.562500,93.911249,28.645833,True,0
9,"0, 1-4, 5-9, 10-14, 15-19, 20-27",93.808050,27.083333,93.292054,32.291667,True,0


## Locations + Mobility

In [104]:
df_loc_mob = pd.read_csv("partition_results_mobility.csv")
df_loc_mob

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
1,"0, 1-9, 10-19, 20-27",96.653144,65.714286,96.653144,66.857143,False,0
2,"0-4, 5-9, 10-14, 15-27",96.044625,56.000000,96.044625,55.428571,False,0
3,base,94.929006,48.000000,94.929006,48.000000,False,0
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",95.436105,44.571429,95.436105,44.000000,False,0
5,"0-4, 5-14, 15-27",96.800826,57.291667,96.800826,56.770833,True,0
6,"0, 1-9, 10-19, 20-27",96.388029,45.833333,96.388029,44.270833,True,0
7,"0-4, 5-9, 10-14, 15-27",95.046440,36.458333,95.046440,32.812500,True,0
8,base,94.427245,29.166667,94.427245,26.562500,True,0
9,"0, 1-4, 5-9, 10-14, 15-19, 20-27",93.911249,34.895833,93.911249,34.375000,True,0


## Locations + Mobility + Phone

In [105]:
df_loc_mob_phone = pd.read_csv("partition_results_mobility_phone.csv")
df_loc_mob_phone

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0
1,"0, 1-9, 10-19, 20-27",96.754564,68.571429,96.653144,65.714286,False,0
2,"0-4, 5-9, 10-14, 15-27",96.146045,58.285714,96.044625,55.428571,False,0
3,base,95.131846,47.428571,94.929006,48.000000,False,0
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",95.638945,42.285714,95.436105,44.000000,False,0
5,"0-4, 5-14, 15-27",96.904025,52.083333,96.800826,56.250000,True,0
6,"0, 1-9, 10-19, 20-27",96.594427,45.833333,96.388029,47.916667,True,0
7,"0-4, 5-9, 10-14, 15-27",95.149639,32.291667,95.046440,32.812500,True,0
8,base,94.633643,26.041667,94.427245,32.812500,True,0
9,"0, 1-4, 5-9, 10-14, 15-19, 20-27",94.220846,31.250000,93.911249,34.375000,True,0


## Comparison

In [106]:
def compare_best_results(feature):
    return pd.concat(
        [
            df_loc[df_loc[feature] == df_loc[feature].max()],
            df_loc_mob[df_loc_mob[feature] == df_loc_mob[feature].max()],
            df_loc_mob_phone[
                df_loc_mob_phone[feature] == df_loc_mob_phone[feature].max()
            ],
        ],
        axis=0,
    )

In [107]:
compare_best_results("train")

,partition,train,test,train (fs),test (fs),SPLIT,TTF
25,"0-4, 5-14, 15-27",96.897810,61.538462,96.624088,64.615385,True,4
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


In [108]:
compare_best_results("test")

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.350365,76.923077,96.076642,73.846154,True,4
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
1,"0, 1-9, 10-19, 20-27",96.754564,68.571429,96.653144,65.714286,False,0


In [109]:
compare_best_results("train (fs)")

,partition,train,test,train (fs),test (fs),SPLIT,TTF
0,"0-4, 5-14, 15-27",96.855984,70.857143,96.653144,71.428571,False,0
0,"0-4, 5-14, 15-27",97.058824,66.857143,97.058824,65.142857,False,0
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


In [110]:
compare_best_results("test (fs)")

,partition,train,test,train (fs),test (fs),SPLIT,TTF
26,"0, 1-9, 10-19, 20-27",96.350365,76.923077,96.076642,73.846154,True,4
26,"0, 1-9, 10-19, 20-27",96.532847,66.153846,96.532847,69.230769,True,4
0,"0-4, 5-14, 15-27",97.160243,65.714286,97.058824,66.857143,False,0


# Add CV


Do everything with

-> Split by the participant, and take one row, cross-validation (10-n), dailies =-> TBD

In [111]:
# Check balanced accuracy instead of raw and plot distributions
# Maybe look at the distr of the RMSE for regression

# Also, maybe use the std. instead of just the mean for features
# Try to use significant cross-validation
# Maybe apply feature selection:
#    * Automatic feature selection
#    * Manual: e.g. greatly reduce the number of location categories except for work and home
# Try to see the effect of some hyperparameters of the random forest

# Use only a few initial weeks in training for some patients
# Maybe try to improve the context with previous information

# Try different targets:
#    * The original sum_phq9 (not the column sum of 9 questions)
#    * Target each question separately
from collections import defaultdict
import cv
from data_processing import *
import data_processing as dp
import metrics

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


def rmse(x, y):
    return np.sqrt(((x - y) ** 2).mean())


def train_cv(
    TYPE="classification",
    TARGET="value",
    SPLIT_BY_PARTICIPANT=False,
    TEST_TAKE_FIRST=0,
    SEED=550,
    N_SPLIT=5,
    return_csv=False,
    use_counts=False,
    feature_selection=False,
    partition=None,
    verbose=False,
    dailies_names=[],
):
    phq9 = dp.load_phq9_targets(
        DATADIR / "df_phq9.csv", type=TYPE, target=TARGET, partition=partition
    )
    locations = load_locations(DATADIR / "df_location_ratio.csv")
    demographics = load_demographics(DATADIR / "df_demographics.csv")

    dailies = [("locations", locations)]

    if "mobility" in dailies_names:
        mobility = load_passive_mobility(DATADIR / "df_passive_mobility_features.csv")
        dailies.append(("mobility", mobility))
    if "phone" in dailies_names:
        phone = load_passive_phone(
            DATADIR / "df_passive_phone_communication_features_brighten_v2.csv"
        )
        dailies.append(("phone", phone))

    combined, merge_result = dp.combine(
        phq9,
        dailies=dailies,
        constants=[demographics],
        prev_phq9=False,
    )
    combined = dp.rf_preprocess(combined)
    if return_csv:
        return combined

    # print('Train set shape:', x_train.shape)
    # print('Test set shape:', x_test.shape)
    # n = len(y_train) + len(y_test)
    # train_pct = len(y_train) / n * 100
    # test_pct = len(y_test) / n * 100
    # print(f'Ratio: {train_pct:.2f}%/{test_pct:.2f}%')
    # print()

    metric_dict = defaultdict(list)
    gen = (
        cv.per_patient_cv(combined, N_SPLIT, TEST_TAKE_FIRST)
        if SPLIT_BY_PARTICIPANT
        else cv.per_row_cv(combined, N_SPLIT)
    )
    for i, (x_train, x_test, y_train, y_test) in enumerate(gen):
        if verbose:
            print(f"--- Split {i} ---")
        if x_train is None:
            print("Skipped, because take first emptied the test set.")
            continue
        if TYPE == "regression":
            model = RandomForestRegressor(
                n_estimators=300, n_jobs=-1, random_state=SEED
            )
        elif TYPE == "classification":
            model = RandomForestClassifier(
                n_estimators=300, n_jobs=-1, random_state=SEED
            )
        model.fit(x_train, y_train)

        train_rmse = rmse(y_train, model.predict(x_train))
        test_rmse = rmse(y_test, model.predict(x_test))
        train_score = model.score(x_train, y_train)
        test_score = model.score(x_test, y_test)

        y_full = np.concatenate((y_train, y_test))
        random_acc = (
            100
            * (
                y_full == np.random.randint(0, y_train.max() + 1, size=y_full.shape)
            ).mean()
        )

        if verbose:
            print(f"Train set RMSE: {train_rmse:.4f}")
            print(f"Test set RMSE:  {test_rmse:.4f}")
            print(f"Train score:", train_score)
            print(f"Test score:", test_score)
            print(f"Random full accuracy: {random_acc:.2f}%")

        # figs, axes = plt.subplots(1, 2, figsize=(16, 6))
        axes = [None, None]

        if verbose:
            print("Train set:")
        train_bal, train_mean = metrics.accuracy_info(
            y_train,
            model.predict(x_train),
            prefix="Training",
            ax=axes[0],
            plot=False,
            verbose=verbose,
        )

        if verbose:
            print("Test set:")
        test_bal, test_mean = metrics.accuracy_info(
            y_test,
            model.predict(x_test),
            prefix="Test",
            ax=axes[1],
            plot=False,
            verbose=verbose,
        )

        (
            train_sel,
            test_sel,
            train_bal_sel,
            train_mean_sel,
            test_bal_sel,
            test_mean_sel,
        ) = (
            feature_selection_results(
                model,
                combined,
                x_train,
                y_train,
                x_test,
                y_test,
                verbose=verbose,
                bal=True,
            )
            if feature_selection == True
            else (None, None)
        )

        metric_dict["train_bal"].append(train_bal * 100)
        metric_dict["train_mean"].append(train_mean * 100)
        metric_dict["test_bal"].append(test_bal * 100)
        metric_dict["test_mean"].append(test_mean * 100)
        metric_dict["train_sel"].append(train_sel)
        metric_dict["test_sel"].append(test_sel)
        metric_dict["train_bal_sel"].append(train_bal_sel * 100)
        metric_dict["test_bal_sel"].append(test_bal_sel * 100)
        metric_dict["train_rand"].append(random_acc)

    for k, v in metric_dict.items():
        metric_dict[k] = np.array(v)

    return metric_dict

In [112]:
def all_combinations_scores_cv(
    dailies_names=[], N_SPLIT=10, TEST_TAKE_FIRST=1, SEED=550, verbose=False
):
    train_bal_scores = []
    train_mean_scores = []
    test_bal_scores = []
    test_mean_scores = []
    train_sel_scores = []
    test_sel_scores = []
    train_bal_sel_scores = []
    test_bal_sel_scores = []
    random_scores = []

    for partition in partitions:
        metric_dict = train_cv(
            TYPE="classification",
            TARGET="value",
            SPLIT_BY_PARTICIPANT=True,
            TEST_TAKE_FIRST=TEST_TAKE_FIRST,
            SEED=SEED,
            N_SPLIT=N_SPLIT,
            return_csv=False,
            use_counts=False,
            feature_selection=True,
            partition=partition,
            verbose=verbose,
            dailies_names=dailies_names,
        )
        train_bal_scores.append(metric_dict["train_bal"].mean())
        train_mean_scores.append(metric_dict["train_mean"].mean())
        test_bal_scores.append(metric_dict["test_bal"].mean())
        test_mean_scores.append(metric_dict["test_mean"].mean())
        train_sel_scores.append(metric_dict["train_sel"].mean())
        test_sel_scores.append(metric_dict["test_sel"].mean())
        train_bal_sel_scores.append(metric_dict["train_bal_sel"].mean())
        test_bal_sel_scores.append(metric_dict["test_bal_sel"].mean())
        random_scores.append(metric_dict["train_rand"].mean())

    df_cv = pd.DataFrame(
        {
            "partition": [
                "0-4, 5-14, 15-27",
                "0, 1-9, 10-19, 20-27",
                "0-4, 5-9, 10-14, 15-27",
                "base",
                "0, 1-4, 5-9, 10-14, 15-19, 20-27",
            ],
            "train": train_mean_scores,
            "test": test_mean_scores,
            "train bal": train_bal_scores,
            "test bal": test_bal_scores,
            "train (fs)": train_sel_scores,
            "test (fs)": test_sel_scores,
            "train bal (fs)": train_bal_sel_scores,
            "test bal (fs)": test_bal_sel_scores,
            "train (rand)": random_scores,
        }
    )
    dailies_substr = "_".join([str(x) for x in dailies_names])
    df_cv.to_csv(
        "partition_results_" + dailies_substr + "_cv" + str(N_SPLIT) + ".csv",
        index=False,
    )

    return df_cv

## Locations (base)

Train: 3m 6s

In [113]:
df_loc_cv = pd.read_csv("partition_results__cv10.csv")
df_loc_cv

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
1,"0, 1-9, 10-19, 20-27",96.460098,57.164210,96.598217,42.589997,96.208622,57.247872,94.711713,42.749504,25.185185
2,"0-4, 5-9, 10-14, 15-27",95.202227,43.204093,95.390149,39.103783,94.615269,45.053297,94.645103,40.442911,24.866494
3,base,94.605833,39.357324,95.000046,38.685655,94.270437,39.922991,94.612196,39.278812,19.888028
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",94.279894,36.642847,95.421229,30.552080,93.609156,37.207199,94.685724,30.583259,17.011197


In [114]:
# Compare CV with no CV
(
    df_loc_cv[["train", "test", "train (fs)", "test (fs)"]].reset_index().iloc[:, 1:]
    - df_loc[df_loc["TTF"] == 1][["train", "test", "train (fs)", "test (fs)"]]
    .reset_index()
    .iloc[:, 1:]
)

,train,test,train (fs),test (fs)
0,0.125161,-4.871878,-0.049930,-7.334385
1,0.116225,-10.621025,0.161191,-6.510517
2,0.142939,-3.104631,-0.048763,-2.597709
3,0.337058,-8.964824,0.396919,-7.056874
4,0.307562,-8.994736,0.130895,-5.074680


In [128]:
comparisons = dict()
for N_SPLIT in range(2, 10):
    loc_cv_= all_combinations_scores_cv(N_SPLIT=N_SPLIT, verbose=False)
    comparisons[N_SPLIT] = (
        loc_cv_[["train", "test", "train (fs)", "test (fs)"]]
        .reset_index()
        .iloc[:, 1:]
        - df_loc[df_loc["TTF"] == 1][["train", "test", "train (fs)", "test (fs)"]]
        .reset_index()
        .iloc[:, 1:]
    )

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

## Locations + Mobility

Train: 4m 26s

In [115]:
df_loc_mob_cv = pd.read_csv("partition_results_mobility_cv10.csv")
df_loc_mob_cv

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
1,"0, 1-9, 10-19, 20-27",96.637095,52.022861,96.829571,35.029912,96.637095,50.289424,96.828364,33.951900,24.788975
2,"0-4, 5-9, 10-14, 15-27",95.379232,42.599205,95.555775,36.933000,95.379232,40.735238,95.584668,35.377746,24.866494
3,base,94.950505,36.383391,95.359242,30.161722,94.950505,36.835017,95.364859,30.510798,19.448751
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",94.456899,35.723454,95.592272,25.356951,94.456899,34.472119,95.567591,24.525994,16.890612


In [116]:
# Compare CV with no CV
(
    df_loc_mob_cv[["train", "test", "train (fs)", "test (fs)"]]
    .reset_index()
    .iloc[:, 1:]
    - df_loc_mob[df_loc_mob["TTF"] == 1][["train", "test", "train (fs)", "test (fs)"]]
    .reset_index()
    .iloc[:, 1:]
)

,train,test,train (fs),test (fs)
0,0.119510,-2.713300,0.119510,-6.699118
1,0.095593,1.016150,0.095593,-0.717288
2,0.122315,0.988467,0.122315,3.822486
3,0.286474,0.141780,0.286474,1.935689
4,0.286938,-1.860439,0.286938,-4.454056


## Locations + Mobility + Phone

Train: 4m 54s

In [117]:
df_loc_mob_phone_cv = pd.read_csv("partition_results_mobility_phone_cv10.csv")
df_loc_mob_phone_cv

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831
1,"0, 1-9, 10-19, 20-27",96.804771,50.734045,97.123562,34.232773,96.637095,50.894828,96.841819,34.390637,24.944014
2,"0-4, 5-9, 10-14, 15-27",95.463083,41.760718,95.612909,36.302858,95.360592,39.254240,95.570225,34.309166,24.728682
3,base,95.118225,36.415122,95.647432,30.176714,94.950505,36.729138,95.365050,30.436445,20.103359
4,"0, 1-4, 5-9, 10-14, 15-19, 20-27",94.708434,34.814304,95.852387,25.079384,94.438268,34.585755,95.535121,24.610732,16.993971


In [118]:
# Compare CV with no CV
(
    df_loc_mob_phone_cv[["train", "test", "train (fs)", "test (fs)"]]
    .reset_index()
    .iloc[:, 1:]
    - df_loc_mob_phone[df_loc_mob_phone["TTF"] == 1][
        ["train", "test", "train (fs)", "test (fs)"]
    ]
    .reset_index()
    .iloc[:, 1:]
)

,train,test,train (fs),test (fs)
0,0.104530,0.940688,0.119510,-4.835011
1,0.065641,-3.628371,0.095593,2.572681
2,0.107352,3.505684,0.103675,2.341488
3,0.256565,-2.511053,0.286474,-0.183614
4,0.242031,-2.769588,0.268308,-4.340420


## Comparison

In [119]:
def compare_best_results_cv(feature):
    return pd.concat(
        [
            df_loc_cv[df_loc_cv[feature] == df_loc_cv[feature].max()],
            df_loc_mob_cv[df_loc_mob_cv[feature] == df_loc_mob_cv[feature].max()],
            df_loc_mob_phone_cv[
                df_loc_mob_phone_cv[feature] == df_loc_mob_phone_cv[feature].max()
            ],
        ],
        axis=0,
    )

In [120]:
compare_best_results_cv("train")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831


In [121]:
compare_best_results_cv("test")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831


In [122]:
compare_best_results_cv("train bal")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
1,"0, 1-9, 10-19, 20-27",96.460098,57.164210,96.598217,42.589997,96.208622,57.247872,94.711713,42.749504,25.185185
1,"0, 1-9, 10-19, 20-27",96.637095,52.022861,96.829571,35.029912,96.637095,50.289424,96.828364,33.951900,24.788975
1,"0, 1-9, 10-19, 20-27",96.804771,50.734045,97.123562,34.232773,96.637095,50.894828,96.841819,34.390637,24.944014


In [123]:
compare_best_results_cv("test bal")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831


In [124]:
compare_best_results_cv("train (fs)")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831


In [125]:
compare_best_results_cv("test (fs)")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831


In [126]:
compare_best_results_cv("train bal (fs)")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
1,"0, 1-9, 10-19, 20-27",96.637095,52.022861,96.829571,35.029912,96.637095,50.289424,96.828364,33.951900,24.788975
1,"0, 1-9, 10-19, 20-27",96.804771,50.734045,97.123562,34.232773,96.637095,50.894828,96.841819,34.390637,24.944014


In [127]:
compare_best_results_cv("test bal (fs)")

,partition,train,test,train bal,test bal,train (fs),test (fs),train bal (fs),test bal (fs),train (rand)
0,"0-4, 5-14, 15-27",96.963106,61.571075,95.861515,48.438016,96.590387,60.450850,95.236918,48.131668,33.738157
0,"0-4, 5-14, 15-27",97.056269,57.018243,95.995625,39.879445,97.056269,55.716989,96.006965,39.486366,33.359173
0,"0-4, 5-14, 15-27",97.140103,57.316527,96.005600,40.514274,97.056269,55.567673,96.006889,39.488637,34.435831
